# MATH 167PS PROJECT
## Names: Le Dao (le.h.dao@sjsu.edu), Dang Minh Nhu Nguyen (dang.m.nguyen@sjsu.edu), Johanna Chen (johanna.chen@sjsu.edu), Andrew Pickard-Christen (andrew.pickard-christen@sjsu.edu), Dr.Matina Bremer

***Loading Data and Import all the packages***

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fancyimpute import IterativeImputer as MICE

store=pd.read_csv('/Users/admin/Downloads/stores.csv')
holiday=pd.read_csv('/Users/admin/Downloads/holidays_events.csv')
transaction=pd.read_csv('/Users/admin/Downloads/transactions.csv')
oil=pd.read_csv('/Users/admin/Downloads/oil.csv')

***Data Exploration***

In [ ]:
print("Information of store.csv\n\n",store.info())
print("Information of holiday_events.csv\n\n",holiday.info())
print("Information of transactions.csv\n\n",transaction.info())
print("Information of oil.csv\n\n",oil.info())

***Data Cleaning***

In [ ]:
#Step 1: Merging the data on the left to reduce data loss
df=store.merge(transaction,on='store_nbr',how='left').merge(holiday,on='date',how='left').merge(oil,how='left')

#Step 2: Rename the data
df=df.rename(columns={'store_nbr':'Store_number','city':'City','state':'State','type_x':'Store_type','cluster':'Cluster','date':'Date','transactions':'Transactions','type_y':'Holiday_type','locale':'Locale','locale_name':'Locale_name','description':'Description','transferred':'Transferred','dcoilwtico':'Oil_price'})

#Step 3: Adding new columns: day, month, year
df['year']=pd.DatetimeIndex(df.Date).year
df['month']=pd.DatetimeIndex(df.Date).month
df['day']=pd.DatetimeIndex(df.Date).day

#Step 4: For loop to fill in NA
for i in list(df.columns.values[1:12]):# Any missing holiday day after merging is a NO holiday/Event
    if df[i].isna().sum() !=0:
        df[i] = df[i].fillna("No Holiday/Event")

#Step 5: Mean imputation for column Oil_price
df['Oil_price']=MICE().fit_transform(df[['year','month','Oil_price']])[:,2]

#Step 6: Check the data information
print(df.info())

#Step 7: Display the first 5 rows
df.head(5)

***GOAL: Is there a trend in the average transaction of 22 cities in different type of holiday events?***

In [ ]:
import plotly.express as px #import plotly to graph a 
df2=pd.DataFrame(df.groupby(['City','Holiday_type']).Transactions.mean()) # Calculate the average transactions of each holiday in different city
df2.reset_index(inplace=True) #reset index level 
#Create a scatter plot where the size of each dot vary differently based on the value of the transaction
fig = px.scatter(df2, x='City', color='Transactions', y='Holiday_type', size='Transactions',
                 color_discrete_sequence=px.colors.qualitative.G10) #Using buil-in quanlitative G10 color to represent discrete data points
fig.update_xaxes(tickangle=45)
fig.show()

***GOAL: What is the highest month that results in the highest average transaction throughout different year?***

In [ ]:
plt.rcParams["figure.figsize"] = (15,8)
df1=pd.DataFrame(df.groupby(['year','month']).Transactions.mean()) #See the average transactions throughout 12 months of the year
df1.reset_index(inplace=True) #reset the index 
fig, ax = plt.subplots()
for i in list(set(df1.year)): #Create a line plot that shows the average transactions of each year
    ax.plot(df1[df1.year==i].month,df1[df1.year==i].Transactions, label=str(i))
plt.legend(bbox_to_anchor=(1,1),fontsize=15)
ax.ticklabel_format(useOffset=False,style='plain') #prevent scientific notation in mathplotlib
ax.set_ylabel('Total Number of Transactions',fontsize=20)
ax.set_xlabel('Month',fontsize=20)
ax.set_xticks(list(set(df1.month))) #adjust the ticks on x-axis so that it shows the value of all 12 months
ax.set_title('Average transaction through years',fontsize=24);
#There's a significant drop in the number of transaction in Agust 2017 since there are no datapoints were updated afterwards
plt.savefig('line.png');

***GOAL: Does the variation in the number of stores in each city affect the trend?***

In [ ]:
import seaborn as sns

df2=df[df.year==2014] #Subset the values n 2014
df2=pd.DataFrame(df2.groupby(['City','Store_number']).Transactions.sum()) # Group by city and store
df2.reset_index(inplace=True) #reset index
df2=df2.drop(columns=["Transactions"]) #drop transactions column
df2=(df2.City).value_counts() #count the number of store in each city
plt.figure(figsize=(25,10)) #make the figure size bigger
palette = sns.color_palette("viridis", 22) #make the color pallete for 22 cities
plt.bar(df2.index,df2.values,color=palette) #graph the number of store in each city
plt.xlabel('City',fontsize=20) #label x axis
plt.xticks(fontsize=19,rotation=45) #make the label on x-axis bigger and rotate 45 degree
plt.yticks(fontsize=19) #make the label on y-axis bigger
plt.ylabel('Number of Stores',fontsize=20) #label y-axis
plt.title("Total Stores in Different City of Ecuador", fontsize=30);

***GOAL: Investigating the store has highest transactions separated by type of Holidays (i.e. Local, Regional, National).***

In [ ]:
#Data Exploration
df3c=df.groupby(['Locale','Cluster']).sum() #we are interested in total of transactions group by cluster and type of holiday
df3c.reset_index(inplace=True) #reset the index so we can call the data frame columns later

#Extracting the data
hol_type=list(df3c.Locale.unique()) #getting the list of holiday type
global national_graph #set global so it could refer back for stacking later on
for h in hol_type: #for loop through all type of holiday
    df3d=df3c[df3c.Locale==h] #subset to another dataframe based on the groupped dataframe
    if list(set(df3d.Locale))==['National']: #check if this is 'National' holiday 
        national_graph=df3d[['Cluster','Transactions']] #save the current df in another data frame with only x and y value
    elif list(set(df3d.Locale))==['Local']: #check if this is 'Local' holiday
        local_graph=df3d[['Cluster','Transactions']] #save the current df in another data frame with only x and y value
    elif list(set(df3d.Locale))==['Regional']: #check if this is 'Regional' holiday
        regional_graph=df3d[['Cluster','Transactions']] #save the current df in another data frame with only x and y value

df3f=pd.merge(national_graph,regional_graph,on='Cluster') 
df3g=df3f.merge(local_graph,on='Cluster') #merging all saved data into a dataframe
df3g=df3g.rename(columns={'Transactions_x':'National','Transactions_y':'Regional','Transactions':'Local'}) #renaming the dataframe

#Plot the data
df3g.plot(x='Cluster', kind='bar', stacked=True,figsize=(15, 10), rot = 0, color=['#f8c291','#0c2461','#079992'],width = 0.8)

plt.xlabel('Cluster Groups',fontdict={'fontsize':16},color='#192a56') #labeling x
plt.ylim(0,5000000) #scale y limit
plt.ylabel('Total Transactions (x10$^6$)',fontdict={'fontsize':16},color='#192a56') #labeling y
plt.title('Total Number of Transaction based on Cluster Groups\n throughout all years separate by type of Holiday',
         fontdict={'fontsize':24},color='#192a56') #set tittle
plt.legend(loc='upper center',ncol=3, fontsize=18) #create legend
plt.show()

***Further Analysis***

In [ ]:
df3e=df[df.Cluster==14] 
df3e1=df3e.groupby(['Store_number','Locale','City']).max()\
.sort_values(by=['Store_number','Transactions'])
out_e=df3e1[['State','Date','Transactions','Locale_name','Description','Oil_price']]
out_e

***GOAL: Does consumer behavior stay consistent during the holiday season and normal working days?***

In [ ]:
# called the file 'df'
plt.figure(figsize=(15,8))
unique_year = df.year.unique() # identified the unique years in the “year” column
unique_year

holiday_trans = [0]*len(unique_year)
non_holiday_trans = [0]*len(unique_year)
counter = 0
for i in unique_year: # loop over each year
    # mask and filter only holidays. compute the sum of all the transactions 
    holiday_trans[counter] = df[(df.Holiday_type!='No Holiday/Event')&(df.Holiday_type!='Work Day')&(df.year==i)].Transactions.sum()
    # mask and filter only non-holidays. compute the sum of all the transactions
    non_holiday_trans[counter] = df[((df.Holiday_type=='No Holiday/Event')|(df.Holiday_type=='Work Day'))&(df.year==i)].Transactions.sum()
    counter += 1

# graph side-by-side bar plot that compares the total transactions by day type (holiday or not) and year
x_pos = np.arange(len(unique_year))
plt.bar(x_pos, holiday_trans, width = -0.35, align = 'edge', label='Holiday')
plt.bar(x_pos, non_holiday_trans, width = 0.35, align = 'edge', label='Non-holiday')
plt.ylabel('Transactions',fontsize=16); 
plt.xlabel('Years',fontsize=16)
plt.title('Total Transactions by Day Type and Year',fontsize=20)
plt.xticks(x_pos, unique_year,fontsize=12) # labels for x-axis
plt.legend(bbox_to_anchor=(1,1));

# check ratio of holidays versus non-holidays
# mask and filter specific dates that are categorized as holidays
test1_hol = df[(df.Holiday_type!='No Holiday/Event')&(df.Holiday_type!='Work Day')].Date.unique()
# mask and filter specific dates that are categorized as non-holidays
test1_nonhol = df[(df.Holiday_type=='No Holiday/Event')|(df.Holiday_type=='Work Day')].Date.unique()
test2_hol = [entry[5:] for entry in test1_hol] # finds the month and day over all data that is considered a holiday
test2_nonhol = [entry[5:] for entry in test1_nonhol] # finds the month and day over all data that is considered NOT a holiday
test3_hol = set(test2_hol) # eliminate repeating dates
print(len(test3_hol)) # 105 holidays recorded in our dataset
test3_nonhol = set(test2_nonhol) # eliminate repeating dates
print(len(test3_nonhol)) # 328 non-holidays recorded in our dataset

***GOAL: Investigate the relationship between the price of crude oil over the year it was sold using a box plot.***

In [ ]:
boxdata=[list(df[df.year==2013].Oil_price),
         list(df[df.year==2014].Oil_price),
         list(df[df.year==2015].Oil_price),
         list(df[df.year==2016].Oil_price),
         list(df[df.year==2017].Oil_price)] # create a list of the five years represented by the data, only providing the oil prices
plt.figure(figsize=(15,8)) # give it a size for visibility
plt.boxplot(boxdata)
plt.title("Boxplot of Oil Prices over Five years", fontsize=18); plt.yticks(fontsize=14)
plt.ylabel("Price of oil per barrel", fontsize=16); plt.xlabel("Year sold", fontsize=16)
plt.xticks(range(1,6),["2013", "2014", "2015", "2016", "2017"], fontsize=13); # rename the tick marks to represent each year
plt.savefig('oil.png');